In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data

In [2]:
df = pd.read_csv('data_20180119_20180420.csv', index_col=0)
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001


In [3]:
import pandas_market_calendars as mcal
from scipy.stats import norm

amzn_t = mcal.get_calendar('NASDAQ')
n_trading = amzn_t.valid_days(start_date='2018-01-02', end_date='2018-12-31')

def bsPrice(S, r, vol, T, K):
    fwd = S * np.exp(r * T)
    stdev = vol * np.sqrt(T)
    d1 = np.log(fwd / K) / stdev + stdev / 2
    d2 = d1 - stdev
    return np.exp(-r * T) * (fwd * norm.cdf(d1) - K * norm.cdf(d2))

In [4]:
#
df['expiry_T1'] = df.apply(lambda x: len(amzn_t.valid_days(start_date=x['t0'], end_date=x['T1'])), axis=1) / len(n_trading)
df['expiry_T2'] = df.apply(lambda x: len(amzn_t.valid_days(start_date=x['t0'], end_date=x['T2'])), axis=1) / len(n_trading)

#
r = 0.0
vol = 0.20

df['C1_BS'] = df.apply(lambda x: bsPrice(x['Adj_S0'], r, vol, x['expiry_T1'], x['K1']), axis=1)
df['C2_BS'] = df.apply(lambda x: bsPrice(x['Adj_S0'], r, vol, x['expiry_T2'], x['K2']), axis=1)

In [5]:
df.head()

,t0,T1,K1,C1,T2,K2,C2,S0,Adj_S0,expiry_T1,expiry_T2,C1_BS,C2_BS
0,2018-01-02,2018-01-19,1000.0,191.525,2018-04-20,800.0,396.725000,59.4505,1189.01001,0.051793,0.302789,189.010842,389.014190
1,2018-01-02,2018-01-19,1100.0,92.525,2018-04-20,1015.0,191.675000,59.4505,1189.01001,0.051793,0.302789,89.939906,178.082515
2,2018-01-02,2018-01-19,1150.0,46.075,2018-04-20,1025.0,183.025000,59.4505,1189.01001,0.051793,0.302789,46.198669,168.987475
3,2018-01-02,2018-01-19,1160.0,38.000,2018-04-20,1050.0,157.776341,59.4505,1189.01001,0.051793,0.302789,38.894821,146.955531
4,2018-01-02,2018-01-19,1170.0,30.450,2018-04-20,1080.0,137.850000,59.4505,1189.01001,0.051793,0.302789,32.249472,122.136522


In [6]:
S0 = df['Adj_S0'].unique()

In [7]:
t0List = list(df.t0.unique())

# dimension: K[k-th t0][t1 and t2][i-th option with i-th strike]
K = [[np.array(df.loc[df.t0 == t0,'K1']),np.array(df.loc[df.t0==t0,'K2'])] for t0 in t0List]

# dimension: Pi[k-th t0][t1 and t2][i-th option with i-th price]
# Pi = [[np.array(df.loc[df.t0 == t0,'C1']),np.array(df.loc[df.t0==t0,'C2'])] for t0 in t0List]
Pi = [[np.array(df.loc[df.t0 == t0,'C1_BS']),np.array(df.loc[df.t0==t0,'C2_BS'])] for t0 in t0List]

In [8]:
# number of t0
N_t0 = len(t0List)

# number of times (t1, t2)
T = 2

# number of options in hedging
N1, N2 = 20, 20
Size = max(N1, N2)

In [9]:
# Assume marginal distribution is uniform
def margDistr(batch_size):
    data = np.random.uniform(size=(N_t0, batch_size, T))
    eps = 500
    for i in range(N_t0):
        for t in range(T):
            data[i,:,t] = (S0[i]-eps) + 2*eps*data[i,:,t]
    return data

In [10]:
BATCH_SIZE = 1000
S = margDistr(BATCH_SIZE)

# different S for different t0: 13*Batch_size*2

In [11]:
Phi = (S[:,:,1]-S[:,:,0])
# define the payoff function of the derivative

### Variables

In [12]:
# parameter: d
d = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'd'
)

In [13]:
# parameter: lambda
Lambda = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, T, Size], dtype=tf.float64),
    trainable = True,
    name = 'lambda'
)

In [14]:
# parameter: Delta_0
Delta_0 = tf.Variable(
    initial_value = tf.random.normal(shape=[N_t0, 1], dtype=tf.float64),
    trainable = True,
    name = 'Delta0'
)

In [15]:
# neural network: Delta(S_1)
Delta_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(BATCH_SIZE,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(N_t0, activation='linear', dtype=tf.float64)
])

### The step-by-step computation of the objective function

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

In [ ]:
hedgePrice = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
    hedgePrice.append(p)

In [ ]:
# take a look
hedgePrice

$d+\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^++\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^++\Delta_{0}\cdot(S_1-S_0)+\Delta_{1}(S_1)\cdot(S_2-S_1)$

In [ ]:
hedgeTerm = []
for n in range(N_t0):
    p = d[n]
    for t in range(T):
        p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
    p += Delta_0[n,0]*(S[n,:,0]-S0[n])
    p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
    hedgeTerm.append(p)

In [ ]:
# take a look
hedgeTerm

$\inf_{h\in\mathcal{H}^m}\int hd\mu_0+\int \beta_\gamma(c-h)d\theta$

$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2} + \Gamma\cdot\left[\left(\Phi(S_1,S_2)-d-\sum_{i=1}^{N_1}\lambda_{i,1}(S_1-K_{i,1})^+-\sum_{i=1}^{N_2}\lambda_{i,2}(S_2-K_{i,2})^+-\Delta_{0}\cdot(S_1-S_0)-\Delta_{1}(S_1)\cdot(S_2-S_1)\right)^+\right]^2$

In [ ]:
# objective function
Price = tf.reduce_mean(hedgePrice, axis=1)

Gamma = 100
diff = Phi - tf.stack(hedgeTerm, axis=0)
Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)

ObjFunc = Price + Gamma*Penalty

In [ ]:
# take a look
Price

In [ ]:
# take a look
Penalty

In [ ]:
# take a look
ObjFunc

### Optimisation

In [16]:
# Combine all the computation as a function
def ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S):
    hedgePrice = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*Pi[n][t][:])
        hedgePrice.append(p)
    
    hedgeTerm = []
    for n in range(N_t0):
        p = d[n]
        for t in range(T):
            p += tf.reduce_sum(Lambda[n,t,:]*tf.maximum(S[n,:,t].reshape(BATCH_SIZE,1)-K[n][t][:],0),axis=1)
        p += Delta_0[n,0]*(S[n,:,0]-S0[n])
        p += Delta_1(S[n,:,0].reshape((1,-1)))[0,n]*(S[n,:,1]-S[n,:,0])
        hedgeTerm.append(p)
    
    Price = tf.reduce_mean(hedgePrice, axis=1)
    Gamma = 100
    diff = Phi-tf.stack(hedgeTerm, axis=0)
    Penalty = tf.reduce_mean(tf.square(tf.nn.relu(diff)), axis=1)
    
    return Price + Gamma*Penalty, Price

In [17]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)

In [18]:
nIter = 10000
threshold = 1e-3
prev_price = tf.ones((N_t0,), dtype = tf.float64) * float('inf')

for step in range(nIter):
    if (step+1) % 500 == 0:
        print("Processing progress: %d / %d" %(step+1, nIter))
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(d)
        tape.watch(Lambda)
        tape.watch(Delta_0)
        for variable in Delta_1.trainable_variables:
            tape.watch(variable)

        obj, price = ObjFunc(d, Lambda, Delta_0, Delta_1, K, Pi, Phi, S)
    gradients_d, gradients_Lambda, gradients_Delta_0 = tape.gradient(obj, [d, Lambda, Delta_0])
    gradients_Delta_1 = tape.gradient(obj, Delta_1.trainable_variables)
    
    optimizer.apply_gradients(
        zip([gradients_d, gradients_Lambda, gradients_Delta_0] + gradients_Delta_1, 
            [d, Lambda, Delta_0] + Delta_1.trainable_variables)
    )
    
    diff = tf.abs(price - prev_price)
    if tf.reduce_max(diff) < threshold:
        print("Converged at step", step+1)
        break
    
    prev_price = price
    
    del tape

Processing progress: 500 / 10000
Processing progress: 1000 / 10000
Processing progress: 1500 / 10000
Processing progress: 2000 / 10000
Processing progress: 2500 / 10000
Processing progress: 3000 / 10000
Processing progress: 3500 / 10000
Processing progress: 4000 / 10000
Processing progress: 4500 / 10000
Processing progress: 5000 / 10000
Processing progress: 5500 / 10000
Processing progress: 6000 / 10000
Processing progress: 6500 / 10000
Processing progress: 7000 / 10000
Processing progress: 7500 / 10000
Processing progress: 8000 / 10000
Processing progress: 8500 / 10000
Processing progress: 9000 / 10000
Processing progress: 9500 / 10000
Processing progress: 10000 / 10000


In [19]:
price

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([815.300518  , 798.47809186, 453.99585122, 977.46799157,
       638.87590624, 500.56631074, 789.31845525, 883.51061642,
       413.76465977, 199.7599196 ,  94.75056727, 561.84750613,
       812.21696848])>

In [20]:
obj

<tf.Tensor: shape=(13,), dtype=float64, numpy=
array([  819.99366413,  5370.94352174,   474.8178309 ,   977.46799157,
       10896.86861182,   659.86642071,  6717.18220542,  2317.0741824 ,
        1027.28549342,  6941.33078497, 15325.00036463,  2716.51547543,
         905.30615014])>